# QLoRA

# installing libraries

In [1]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets
!pip install trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 16.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 58.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This

# Loading dataset

In [19]:
from datasets import load_dataset, Dataset, DatasetDict
from dataclasses import dataclass, field
from typing import Optional
import torch
from peft import LoraConfig
from tqdm import tqdm
import pandas as pd
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, HfArgumentParser, TrainingArguments, AutoTokenizer, pipeline
from trl import SFTTrainer

tqdm.pandas()

# nli_data = load_dataset('klue', 'nli')

# print(nli_data)

# train_data = nli_data['train']
# validation_data = nli_data['validation']


train_data = pd.read_csv('train.csv', sep='\t')
validation_data = pd.read_csv('validation.csv', sep='\t')
dataset_dict = DatasetDict({
    'train': train_data,
    'validation': validation_data
})
train_data = dataset_dict['train']
print(train_data)
validation_data = dataset_dict['validation']
print(validation_data)

                                                   text     label
0     The production is to be liquidated before June...  negative
1     The market making in accordance with the agree...   neutral
2     The largest construction company in Finland , ...   neutral
3     The transaction , which includes US$ 1.5 billi...   neutral
4     MADISON , Wis. , Feb. 6 - PRNewswire - -- Fisk...  positive
...                                                 ...       ...
3401  Based on the first quarter result , existing o...  negative
3402  Aldata said that there are still a number of o...   neutral
3403  The casing comprises a first side casing membe...   neutral
3404  The most significant capital expenditure items...   neutral
3405  Stockmann MasterCard has widened the scope of ...  positive

[3406 rows x 2 columns]
                                                  text     label
0    According to Scanfil , demand for telecommunic...  negative
1    Kemira 's partner in the project is St. Peters..

# Processing dataset

In [37]:
import random


# id_to_label = {0:'entailment', 1:'neutral', 2:'contradiction'}

# question_template = "### Human: 다음 두 문장의 관계를 entailment, neutral, contradiction 중 하나로 분류해줘. "
# train_instructions = [f'{question_template}\npremise: {x}\nhypothesis: {y}\n\n### Assistant: {id_to_label[z]}' for x,y,z in zip(train_data['premise'],train_data['hypothesis'],train_data['label'])]
# validation_instructions = [f'{question_template}\npremise: {x}\nhypothesis: {y}\n\n### Assistant: {id_to_label[z]}' for x,y,z in zip(validation_data['premise'],validation_data['hypothesis'],validation_data['label'])]

id_to_label = {'negative':0, 'neutral':1, 'positive':2}

question_template = "### Classify the sentiment of the following sentence as negative, neutral."

train_instructions = [f'{question_template}\nsentence: {x}\n\n### Answer: {y}' for x,y in zip(train_data['text'],train_data['label'])]
validation_instructions = [f'{question_template}\npremise: {x}\n\n### Answer: {y}' for x,y in zip(validation_data['text'],validation_data['label'])]


# random.seed(42)
# train_instructions = random.sample(train_instructions, len(train_instructions))
# train_instructions = train_instructions[:300]
# random.seed(42)
# validation_instructions = random.sample(validation_instructions, len(validation_instructions))
# # validation_instructions = validation_instructions[:100]


ds_train = Dataset.from_dict({"text": train_instructions})
ds_validation = Dataset.from_dict({"text": validation_instructions})
instructions_ds_dict = DatasetDict({"train": ds_train, "eval": ds_validation})

In [38]:
print(len(train_instructions))

3406


In [39]:
instructions_ds_dict['train']['text'][0]

'### Classify the sentiment of the following sentence as negative, neutral.\nsentence: The production is to be liquidated before June 2009 and 325 employees loose their jobs\n\n### Answer: negative'

In [40]:
instructions_ds_dict['eval']['text'][0]

'### Classify the sentiment of the following sentence as negative, neutral.\npremise: According to Scanfil , demand for telecommunications network products has fluctuated significantly in the third quarter of 2006 , and the situation is expected to remain unstable for the rest of the year\n\n### Answer: negative'

# use beomi version pre-trained llama2

In [41]:
model_name = "beomi/llama-2-ko-7b"


@dataclass
class ScriptArguments:
    model_name: Optional[str] = field(default=model_name, metadata={"help": "the model name"})
    dataset_text_field: Optional[str] = field(default="text", metadata={"help": "the text field of the dataset"})
    log_with: Optional[str] = field(default=None, metadata={"help": "use 'wandb' to log with wandb"})
    learning_rate: Optional[float] = field(default=1.41e-5, metadata={"help": "the learning rate"})
    batch_size: Optional[int] = field(default=4, metadata={"help": "the batch size"})
    seq_length: Optional[int] = field(default=512, metadata={"help": "Input sequence length"})
    gradient_accumulation_steps: Optional[int] = field(
        default=2, metadata={"help": "the number of gradient accumulation steps"}
    )
    load_in_8bit: Optional[bool] = field(default=False, metadata={"help": "load the model in 8 bits precision"})
    load_in_4bit: Optional[bool] = field(default=True, metadata={"help": "load the model in 4 bits precision"})
    use_peft: Optional[bool] = field(default=True, metadata={"help": "Wether to use PEFT or not to train adapters"})
    trust_remote_code: Optional[bool] = field(default=True, metadata={"help": "Enable `trust_remote_code`"})
    output_dir: Optional[str] = field(default="output", metadata={"help": "the output directory"})
    peft_lora_r: Optional[int] = field(default=64, metadata={"help": "the r parameter of the LoRA adapters"})
    peft_lora_alpha: Optional[int] = field(default=16, metadata={"help": "the alpha parameter of the LoRA adapters"})
    logging_steps: Optional[int] = field(default=1, metadata={"help": "the number of logging steps"})
    use_auth_token: Optional[bool] = field(default=False, metadata={"help": "Use HF auth token to access the model"})
    num_train_epochs: Optional[int] = field(default=1, metadata={"help": "the number of training epochs"}) #train epoch
    max_steps: Optional[int] = field(default=-1, metadata={"help": "the number of training steps"})
    save_steps: Optional[int] = field(
        default=100, metadata={"help": "Number of updates steps before two checkpoint saves"}
    )
    save_total_limit: Optional[int] = field(default=10, metadata={"help": "Limits total number of checkpoints."})
    push_to_hub: Optional[bool] = field(default=False, metadata={"help": "Push the model to HF Hub"})
    hub_model_id: Optional[str] = field(default=None, metadata={"help": "The name of the model on HF Hub"})


script_args = ScriptArguments()

# implement quantization

In [42]:
if script_args.load_in_8bit and script_args.load_in_4bit:
    raise ValueError("You can't load the model in 8 bits and 4 bits at the same time")
elif script_args.load_in_8bit or script_args.load_in_4bit:
    quantization_config = BitsAndBytesConfig(
        load_in_8bit=script_args.load_in_8bit, load_in_4bit=script_args.load_in_4bit
    )
    device_map = {"": 0}
    torch_dtype = torch.bfloat16
else:
    device_map = None
    quantization_config = None
    torch_dtype = None

model = AutoModelForCausalLM.from_pretrained(
    script_args.model_name,
    quantization_config=quantization_config,
    device_map=device_map,
    trust_remote_code=script_args.trust_remote_code,
    torch_dtype=torch_dtype,
    use_auth_token=script_args.use_auth_token,
)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:469: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

# QLora

In [43]:
dataset = instructions_ds_dict

training_args = TrainingArguments(
    output_dir=script_args.output_dir,
    per_device_train_batch_size=script_args.batch_size,
    gradient_accumulation_steps=script_args.gradient_accumulation_steps,
    learning_rate=script_args.learning_rate,
    logging_steps=script_args.logging_steps,
    num_train_epochs=script_args.num_train_epochs,
    max_steps=script_args.max_steps,
    report_to=script_args.log_with,
    save_steps=script_args.save_steps,
    save_total_limit=script_args.save_total_limit,
    push_to_hub=script_args.push_to_hub,
    hub_model_id=script_args.hub_model_id,
)

if script_args.use_peft:
    peft_config = LoraConfig(
        r=script_args.peft_lora_r,
        lora_alpha=script_args.peft_lora_alpha,
        bias="none",
        task_type="CAUSAL_LM",
    )
else:
    peft_config = None

trainer = SFTTrainer(
    model=model,
    args=training_args,
    max_seq_length=script_args.seq_length,
    train_dataset=dataset['train'],
    eval_dataset=dataset['eval'],
    dataset_text_field=script_args.dataset_text_field,
    peft_config=peft_config,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length, dataset_text_field. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:321: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/3406 [00:00<?, ? examples/s]

Map:   0%|          | 0/378 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:407: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


# Train on colab

In [44]:
trainer.train()

trainer.save_model(training_args.output_dir)

Step,Training Loss
1,4.461700
2,4.927400
3,5.281700
4,4.931900
5,4.848200
6,4.845300
7,5.022400
8,4.825400
9,5.179100
10,5.069300


In [49]:
trainer.predict(instructions_ds_dict['eval']['text'][1])

AttributeError: 'str' object has no attribute 'size'

# Evaluate the model

In [52]:
model.eval()
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, HfArgumentParser, TrainingArguments, AutoTokenizer, pipeline
tokenizer = AutoTokenizer.from_pretrained(model_name)

pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    device_map={'':0},
)

In [53]:
print(instructions_ds_dict['eval']['text'][1])
print(instructions_ds_dict['eval']['text'][1].split('### Answer: ')[0] + '### Answer:')

### Classify the sentiment of the following sentence as negative, neutral.
premise: Kemira 's partner in the project is St. Petersburg Water Works

### Answer: neutral
### Classify the sentiment of the following sentence as negative, neutral.
premise: Kemira 's partner in the project is St. Petersburg Water Works

### Answer:


In [54]:

query = instructions_ds_dict['eval']['text'][1].split('### Answer: ')[0] + '### Answer:'
print(query)
queries = [instructions_ds_dict['eval']['text'][i].split('### Answer: ')[0] + '### Answer:' for i in range(len(instructions_ds_dict['eval']))]
print(queries)
sequences = pipeline(
    queries,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_new_tokens=3,
    early_stopping=True,
    # do_sample=True,
)

### Classify the sentiment of the following sentence as negative, neutral.
premise: Kemira 's partner in the project is St. Petersburg Water Works

### Answer:
['### Classify the sentiment of the following sentence as negative, neutral.\npremise: According to Scanfil , demand for telecommunications network products has fluctuated significantly in the third quarter of 2006 , and the situation is expected to remain unstable for the rest of the year\n\n### Answer:', "### Classify the sentiment of the following sentence as negative, neutral.\npremise: Kemira 's partner in the project is St. Petersburg Water Works\n\n### Answer:", '### Classify the sentiment of the following sentence as negative, neutral.\npremise: Finnish Rautaruukki has been awarded a contract to supply and install steel superstructures for the Partihallsf+rbindelsen bridge in Gothenburg in Sweden\n\n### Answer:', '### Classify the sentiment of the following sentence as negative, neutral.\npremise: It is also 7.7 pct abov

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:611: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


In [55]:
results = []

for seq in sequences:
  # print(seq)
  result = seq[0]['generated_text'].split('### Answer:')[1]
  results.append(result)

labels = []
instructions_ds_dict['eval']['text'][0]
print(instructions_ds_dict['eval']['text'][0])
for label in instructions_ds_dict['eval']['text']:
  result = label.split('### Answer:')[1]
  labels.append(result)

print("Accuracy: ", (len([1 for x, y in zip(results, labels) if y in x]) / len(labels)))

### Classify the sentiment of the following sentence as negative, neutral.
premise: According to Scanfil , demand for telecommunications network products has fluctuated significantly in the third quarter of 2006 , and the situation is expected to remain unstable for the rest of the year

### Answer: negative
Accuracy:  0.6005291005291006


In [56]:
print(len(instructions_ds_dict['eval']))

378


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [59]:
output_merged_dir = "/content/drive/MyDrive/merged_result/news_classification_llama2_7b/final_merged_checkpoint"

In [58]:

model.save_pretrained( "/content/drive/MyDrive/merged_result/news_classification_llama2_7b/final_merged_checkpoint")

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2577: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 4096}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


In [60]:
tokenizer.save_pretrained(output_merged_dir)

('/content/drive/MyDrive/merged_result/news_classification_llama2_7b/final_merged_checkpoint/tokenizer_config.json',
 '/content/drive/MyDrive/merged_result/news_classification_llama2_7b/final_merged_checkpoint/special_tokens_map.json',
 '/content/drive/MyDrive/merged_result/news_classification_llama2_7b/final_merged_checkpoint/tokenizer.json')

In [ ]:
# nli_data = load_dataset('klue', 'nli')

# print(nli_data)

# train_data = nli_data['train']
# validation_data = nli_data['validation']
# id_to_label = {0:'entailment', 1:'neutral', 2:'contradiction'}

# question_template = "### Human: 다음 두 문장의 관계를 entailment, neutral, contradiction 중 하나로 분류해줘. "
# train_instructions = [f'{question_template}\npremise: {x}\nhypothesis: {y}\n\n### Assistant: {id_to_label[z]}' for x,y,z in zip(train_data['premise'],train_data['hypothesis'],train_data['label'])]
# validation_instructions = [f'{question_template}\npremise: {x}\nhypothesis: {y}\n\n### Assistant: {id_to_label[z]}' for x,y,z in zip(validation_data['premise'],validation_data['hypothesis'],validation_data['label'])]
# ds_train = Dataset.from_dict({"text": train_instructions})
# ds_validation = Dataset.from_dict({"text": validation_instructions})
# instructions_ds_dict = DatasetDict({"train": ds_train, "eval": ds_validation})
# print(instructions_ds_dict['eval']['text'][1])
# print(instructions_ds_dict['eval']['text'][1].split('### Assistant: ')[0] + '### Assistant:')